### Protein. Bone Marrow (BM) + Spleen (SPL) 
### ADT QC steps + DSB + filtering clusters + differentially expressed proteins
---
##### mq224

Date: 2021-01-19

In [1]:
#SETTING ENVIRONMENT 
import rpy2

import numpy as np
import pandas as pd
import scanpy as sc
import os.path

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, dpi_save=1000)

/usr/local/lib/python3.7/dist-packages/numba/core/cpu.py:77: UserWarning: Numba extension module 'sparse._numba_extension' failed to load due to 'ModuleNotFoundError(No module named 'numba.targets')'.
  numba.core.entrypoints.init_all()


In [2]:
from datetime import datetime
now = datetime.now()
prefix = now.strftime('%Y%m%d')
print(prefix)

20210222


In [3]:
from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

In [4]:
#this file hasn't been dsb normalised. 
output = sc.read("h5ad_part2/20210119_Laurenti_mq224_ADT_postQC_filtered_BM_SPL_SIGAB11_SIGAD11.h5ad")

# HSC/MPP only - Differentially expressed proteins

In [5]:
import pandas as pd
new_metadata=pd.read_csv("../../../Data/20210129_COMBO10_SAM_DOD4_metadata.txt",delimiter="\t")

In [6]:
new_metadata

,Unnamed: 0,donor,organ,kmeans2,leiden.1.2
0,_04_AAACCCAAGATGTTCC.1.1,DOD4,BM,0,5
1,_04_AAACCCACAATACAGA.1.1,DOD4,BM,0,5
2,_04_AAACCCAGTAAGATAC.1.1,DOD4,BM,0,4
3,_04_AAACCCATCCTAAACG.1.1,DOD4,BM,0,5
4,_04_AAACCCATCGTTACCC.1.1,DOD4,BM,0,5
...,...,...,...,...,...
9374,_04_TTTGTTGGTAAGGCTG.1.3,DOD4,SPL,1,0
9375,_04_TTTGTTGGTATGTGTC.1.3,DOD4,SPL,1,11
9376,_04_TTTGTTGGTCCTGTTC.1.3,DOD4,SPL,1,0
9377,_04_TTTGTTGGTGACACGA.1.3,DOD4,SPL,1,4


In [7]:
output.obs_names

Index(['TCCCATGAGTCATGGG-0', 'TTGATGGAGAAACTCA-0', 'ATCATTCCACCATATG-0',
       'TCTATACCAGCTATTG-0', 'GGGCCATCAAATGGAT-0', 'CAACGGCCACTCCGGA-0',
       'ACGGTCGCATAATGCC-0', 'CACAGATGTCGCAACC-0', 'AAGACAAAGCCTCTCT-0',
       'CGGGCATTCAAGGCTT-0',
       ...
       'TGCTCCATCAACGCTA-1', 'AGATCCATCGTGTGGC-1', 'TTGCATTTCCATCTCG-1',
       'TCCATGCAGCCATTCA-1', 'TAAGCACAGCGAGTCA-1', 'GGAGGTAAGGTTCATC-1',
       'GCCAGCATCAGCGCGT-1', 'TGCGGCAGTGATGAAT-1', 'TAGGTTGAGATGTTCC-1',
       'CAAGAGGCAGTCGAGA-1'],
      dtype='object', length=29463)

In [8]:
new_metadata = pd.DataFrame(new_metadata)
new_metadata.index = new_metadata["Unnamed: 0"]
new_metadata["Tissue"] = new_metadata["organ"]  
meta_SPL = new_metadata.groupby(new_metadata.Tissue)
meta_SPL = meta_SPL.get_group("SPL")
meta_BM = new_metadata.groupby(new_metadata.Tissue)
meta_BM = meta_BM.get_group("BM")

In [9]:
meta_SPL

,Unnamed: 0,donor,organ,kmeans2,leiden.1.2,Tissue
Unnamed: 0,,,,,,
_04_AAACCCAAGAACGTGC.1.3,_04_AAACCCAAGAACGTGC.1.3,DOD4,SPL,1,0,SPL
_04_AAACCCAAGCGCTGCT.1.3,_04_AAACCCAAGCGCTGCT.1.3,DOD4,SPL,1,0,SPL
_04_AAACCCAAGGAACGTC.1.3,_04_AAACCCAAGGAACGTC.1.3,DOD4,SPL,1,11,SPL
_04_AAACCCACAAGCTCTA.1.3,_04_AAACCCACAAGCTCTA.1.3,DOD4,SPL,1,0,SPL
_04_AAACCCAGTGCAACAG.1.3,_04_AAACCCAGTGCAACAG.1.3,DOD4,SPL,1,0,SPL
...,...,...,...,...,...,...
_04_TTTGTTGGTAAGGCTG.1.3,_04_TTTGTTGGTAAGGCTG.1.3,DOD4,SPL,1,0,SPL
_04_TTTGTTGGTATGTGTC.1.3,_04_TTTGTTGGTATGTGTC.1.3,DOD4,SPL,1,11,SPL
_04_TTTGTTGGTCCTGTTC.1.3,_04_TTTGTTGGTCCTGTTC.1.3,DOD4,SPL,1,0,SPL


In [10]:
meta_BM

,Unnamed: 0,donor,organ,kmeans2,leiden.1.2,Tissue
Unnamed: 0,,,,,,
_04_AAACCCAAGATGTTCC.1.1,_04_AAACCCAAGATGTTCC.1.1,DOD4,BM,0,5,BM
_04_AAACCCACAATACAGA.1.1,_04_AAACCCACAATACAGA.1.1,DOD4,BM,0,5,BM
_04_AAACCCAGTAAGATAC.1.1,_04_AAACCCAGTAAGATAC.1.1,DOD4,BM,0,4,BM
_04_AAACCCATCCTAAACG.1.1,_04_AAACCCATCCTAAACG.1.1,DOD4,BM,0,5,BM
_04_AAACCCATCGTTACCC.1.1,_04_AAACCCATCGTTACCC.1.1,DOD4,BM,0,5,BM
...,...,...,...,...,...,...
_04_TTTGGTTGTACTCCCT.1.1,_04_TTTGGTTGTACTCCCT.1.1,DOD4,BM,0,5,BM
_04_TTTGGTTGTATATGGA.1.1,_04_TTTGGTTGTATATGGA.1.1,DOD4,BM,0,4,BM
_04_TTTGGTTGTTGACTGT.1.1,_04_TTTGGTTGTTGACTGT.1.1,DOD4,BM,0,4,BM


In [11]:
cells1 = pd.Index(meta_BM["Unnamed: 0"]).astype(str)
cells2 = pd.Index(meta_SPL["Unnamed: 0"]).astype(str)

In [12]:
#for BM
new1 =[]
for cell in cells1:
    cell = cell.replace('_04_',"")
    cell = cell.replace('.1',"")
    new1.append(cell)
    
#new1 = pd.Index([x[:-2] for x in new1])
new1 = pd.Index([x+"-0" for x in new1])

In [13]:
#for SPL
new2 =[]
for cell in cells2:
    cell = cell.replace('_04_',"")
    cell = cell.replace('.1',"")
    new2.append(cell)

new2 = pd.Index([x[:-2] for x in new2])
new2 = pd.Index([x+"-1" for x in new2])

In [14]:
meta_BM.index = new1
meta_SPL.index= new2

In [15]:
meta_BM_SPL= pd.concat([meta_BM, meta_SPL], join="inner")

In [16]:
# get the overlapping genes between mRNA and AB
OL1 = np.intersect1d(meta_BM_SPL.index, output.obs_names)
print(len(OL1))

9216


In [17]:
output = output[OL1,:].copy()

In [18]:
meta_BM_SPL = meta_BM_SPL.loc[OL1,:]

In [19]:
output.obs

,Tissue,batch,n_genes,n_counts,leiden
AAACCCAAGAACGTGC-1,SPL,1,55,461.0,0
AAACCCAAGATGTTCC-0,BM,0,48,191.0,7
AAACCCAAGCGCTGCT-1,SPL,1,64,359.0,0
AAACCCAAGGAACGTC-1,SPL,1,66,505.0,2
AAACCCACAAGCTCTA-1,SPL,1,65,437.0,2
...,...,...,...,...,...
TTTGTTGGTATGTGTC-1,SPL,1,59,418.0,0
TTTGTTGGTCCTGTTC-1,SPL,1,63,370.0,0
TTTGTTGGTGACACGA-1,SPL,1,66,373.0,0
TTTGTTGGTGCATGAG-1,SPL,1,64,432.0,0


In [20]:
output.obs["donor"]=meta_BM_SPL[["donor"]]
output.obs["kmeans2"]=meta_BM_SPL[["kmeans2"]]

In [21]:
output

AnnData object with n_obs × n_vars = 9216 × 198
    obs: 'Tissue', 'batch', 'n_genes', 'n_counts', 'leiden', 'donor', 'kmeans2'
    var: 'n_cells'

In [22]:
output.obs

,Tissue,batch,n_genes,n_counts,leiden,donor,kmeans2
AAACCCAAGAACGTGC-1,SPL,1,55,461.0,0,DOD4,1
AAACCCAAGATGTTCC-0,BM,0,48,191.0,7,DOD4,0
AAACCCAAGCGCTGCT-1,SPL,1,64,359.0,0,DOD4,1
AAACCCAAGGAACGTC-1,SPL,1,66,505.0,2,DOD4,1
AAACCCACAAGCTCTA-1,SPL,1,65,437.0,2,DOD4,1
...,...,...,...,...,...,...,...
TTTGTTGGTATGTGTC-1,SPL,1,59,418.0,0,DOD4,1
TTTGTTGGTCCTGTTC-1,SPL,1,63,370.0,0,DOD4,1
TTTGTTGGTGACACGA-1,SPL,1,66,373.0,0,DOD4,1
TTTGTTGGTGCATGAG-1,SPL,1,64,432.0,0,DOD4,1


In [23]:
output.obs["kmeans2"] = output.obs["kmeans2"].astype("category")

In [24]:
output.obs["kmeans2"]

AAACCCAAGAACGTGC-1    1
AAACCCAAGATGTTCC-0    0
AAACCCAAGCGCTGCT-1    1
AAACCCAAGGAACGTC-1    1
AAACCCACAAGCTCTA-1    1
                     ..
TTTGTTGGTATGTGTC-1    1
TTTGTTGGTCCTGTTC-1    1
TTTGTTGGTGACACGA-1    1
TTTGTTGGTGCATGAG-1    1
TTTGTTGTCTGTTCAT-0    0
Name: kmeans2, Length: 9216, dtype: category
Categories (2, int64): [0, 1]

In [25]:
output.obs["kmeans2"].cat.categories.tolist()

[0, 1]

In [26]:
def save_dex_results(adata, key, grouping, basename, outpath, prefix):
    
    statcols =['antibody_name', 'scores', 'logfoldchanges','pvals', 'pvals_adj' ]
    
    groups = adata.obs[grouping].unique() 
    groups=groups.astype("str")
    n = adata.shape[1]
    
    marker_antibody_full = {}   
    for g in groups:
        marker_antibody = np.reshape( np.array(adata.uns[key]['names'][g]), (n,1) )
       
        marker_antibody_full.setdefault(g, None )
        marker_antibody_full[g] = np.reshape( np.array(adata.uns[key]['names'][g]), (n,1) )
        marker_antibody_full[g] = np.concatenate( [marker_antibody_full[g], 
                                                   np.reshape(adata.uns[key]['scores'][g], (n,1) ) ], axis=1)
        marker_antibody_full[g] = np.concatenate( [marker_antibody_full[g], 
                                                   np.reshape(adata.uns[key]['logfoldchanges'][g], (n,1) ) ], axis=1)
        marker_antibody_full[g] = np.concatenate( [marker_antibody_full[g], 
                                                   np.reshape(adata.uns[key]['pvals'][g], (n,1) ) ], axis=1)
        marker_antibody_full[g] = np.concatenate( [marker_antibody_full[g], 
                                                   np.reshape(adata.uns[key]['pvals_adj'][g], (n,1) ) ], axis=1)   
    for c in marker_antibody_full.keys():
        temp = pd.DataFrame(marker_antibody_full[c], columns=statcols)   
        temp.to_csv( os.path.join(outpath,  
                    prefix+basename+key+'_group_'+c+'.txt'), 
                    index=None, sep='\t')
                    
                   
def export_top_n_diff_antibody(adata, key, groups, n, basename, outpath, prefix):
    groups=groups.astype("str") 
    marker_antibody = np.reshape(np.array(adata.uns[key]['names'][groups[0]][:n]  ), (n,1) )
    for c in groups[1:]:
        marker_antibody = np.concatenate( [marker_antibody, np.reshape(adata.uns[key]['names'][c][:n], (n,1) ) ], axis=1)
    markers_frame = pd.DataFrame(marker_antibody, columns=groups )
    markers_frame.to_csv(os.path.join(outpath,
                         prefix+basename+key+'_top'+str(n)+'_antibody_markers.txt'), 
                         index=None, sep='\t')
                         
def bundled_DEx_shenanigans(adata, group, key, bname, outpath, prefix, n_top_markers=198):
        
    sc.tl.rank_genes_groups(adata, 
                        groupby=group,
                        key_added=key,
                        method='wilcoxon', 
                        tie_correct=True,     
                        use_raw=False)
    
    save_dex_results(adata, key, group, bname, outpath, prefix)
    targets = adata.obs[group].unique()
    export_top_n_diff_antibody(adata, key, targets, n_top_markers, bname, outpath, prefix)
    
    

In [27]:
bundled_DEx_shenanigans(adata=output,
                        group='kmeans2',
                        key='-HSCMPP_DEP',  
                        bname='_BM-SPL',
                        outpath="./DEG_part4B/",
                        prefix=prefix)

ranking genes


... storing 'donor' as categorical


    finished: added to `.uns['-HSCMPP_DEP']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


In [29]:
# TEST
sc.tl.rank_genes_groups(output, 'kmeans2', method='wilcoxon')
result =output.uns['rank_genes_groups']
groups = result['names'].dtype.names
df=pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores', 'logfoldchanges','pvals', 'pvals_adj']})

ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


In [30]:
df

,0_n,0_s,0_l,0_p,1_n,1_s,1_l,1_p
0,CD158,21.789307,3.313308,5.802365e-104,CD107a,46.147831,7.126022,0.000000e+00
1,CD370,16.181637,1.248352,8.971264e-58,CD99,38.967236,7.026029,0.000000e+00
2,CD184,15.934348,1.520596,4.529217e-56,CD82,31.773813,6.722711,9.828994e-220
3,CD69,15.314118,1.583880,6.769438e-52,Folate,27.393200,3.032251,1.635984e-163
4,CD110,13.305926,1.085764,2.016085e-39,CD102,26.825644,6.100643,6.427472e-157
...,...,...,...,...,...,...,...,...
193,CD102,-26.825644,-6.100643,6.427472e-157,CD110,-13.305926,-1.085764,2.016085e-39
194,Folate,-27.393200,-3.032251,1.635984e-163,CD69,-15.314118,-1.583880,6.769438e-52
195,CD82,-31.773813,-6.722711,9.828994e-220,CD184,-15.934348,-1.520596,4.529217e-56
196,CD99,-38.967236,-7.026029,0.000000e+00,CD370,-16.181637,-1.248352,8.971264e-58


In [31]:
np.mean(output.X[:,output.var_names=="CD158"])
np.mean(output.X[:,output.var_names=="CD158"])

0.19021267

In [32]:
output_0= output[output.obs["kmeans2"]==0,:].copy()
output_1= output[output.obs["kmeans2"]==1,:].copy()



In [33]:
output_0

AnnData object with n_obs × n_vars = 3714 × 198
    obs: 'Tissue', 'batch', 'n_genes', 'n_counts', 'leiden', 'donor', 'kmeans2'
    var: 'n_cells'
    uns: '-HSCMPP_DEP', 'rank_genes_groups'

In [34]:
print(np.mean(output_0.X[:,output.var_names=="CD184"]))
print(np.mean(output_1.X[:,output.var_names=="CD184"]))

0.7081313
0.30679756


In [35]:
print(np.mean(output_0.X[:,output.var_names=="CD158"]))
print(np.mean(output_1.X[:,output.var_names=="CD158"]))

0.40037695
0.048346058


In [36]:
print(np.mean(output_0.X[:,output.var_names=="Folate"]))
print(np.mean(output_1.X[:,output.var_names=="Folate"]))

0.23317178
1.1468557


In [37]:
print(np.mean(output_0.X[:,output.var_names=="CD370"]))
print(np.mean(output_1.X[:,output.var_names=="CD370"]))

0.6567043
0.32988006
